In [3]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.stats import norm

df = pd.read_csv('user_info.csv')

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("../assets/chromedriver", options=options)

In [4]:
class Player:
    def __init__(self, name, level, rating, prestige, games_won, qps, medals, hero):
        self.name = name
        self.level = level
        self.rating = rating
        self.prestige = prestige
        self.qps = qps
        self.medals = medals
        self.hero = hero
        self.kd_ratio = [i/(1+sum([qps.elims,qps.deaths])) for i in [qps.elims,qps.deaths]]
        self.games_won = games_won

class Stats:
    def __init__(self, elims=0, dmg_done=0, deaths=0, solo_kills=0):
        self.elims = elims
        self.dmg_done = dmg_done
        self.deaths = deaths
        self.solo_kills = solo_kills
        
class Medals:
    def __init__(self, bronze=0, silver=0, gold=0):
        self.bronze = bronze
        self.silver = silver
        self.gold = gold
        
hero_list = ['ana','ashe','baptiste','bastion','brigitte','dVa','doomfist',
             'genji','hanzo','junkrat','lucio','mccree','mei','mercy','moira',
             'orisa','pharah','reaper','reinhardt','roadhog','soldier76','sombra',
             'symmetra','torbjorn','tracer','widowmaker','winston','wreckingBall',
             'zarya','zenyatta','sigma']

def create_player(js):
    heroes = {}
    if 'quickPlayStats' not in js:
        for hero in hero_list:
            heroes.update({hero: Stats(0,0,0,0)})
        return Player(js['name'], js['level'],js['rating'],js['prestige'], 0, Stats(), Medals(), heroes)
    if 'careerStats' not in js['quickPlayStats']:
        for hero in hero_list:
            heroes.update({hero: Stats(0,0,0,0)})
        return Player(js['name'], js['level'],js['rating'],js['prestige'], 0, Stats(), Medals(), heroes)
    if js.get('quickPlayStats',{}).get('careerStats',{}) == None or 'allHeroes' not in js.get('quickPlayStats',{}).get('careerStats',{}):
        for hero in hero_list:
            heroes.update({hero: Stats(0,0,0,0)})
        return Player(js['name'], js['level'],js['rating'],js['prestige'], 0, Stats(), Medals(), heroes)
    
    elims = 0
    damageDone = 0
    deaths = 0
    soloKills = 0

    if js['quickPlayStats']['careerStats']['allHeroes']['combat'] != None:

        if 'eliminations' in js['quickPlayStats']['careerStats']['allHeroes']['combat']:
            elims = js['quickPlayStats']['careerStats']['allHeroes']['combat']['eliminations']

        if 'damageDone' in js['quickPlayStats']['careerStats']['allHeroes']['combat']:
            damageDone = js['quickPlayStats']['careerStats']['allHeroes']['combat']['damageDone']

        if 'deaths' in js['quickPlayStats']['careerStats']['allHeroes']['combat']:
            deaths = js['quickPlayStats']['careerStats']['allHeroes']['combat']['deaths']

        if 'soloKills' in js['quickPlayStats']['careerStats']['allHeroes']['combat']:
            soloKills = js['quickPlayStats']['careerStats']['allHeroes']['combat']['soloKills']
    
    qps = Stats(elims,damageDone,deaths,soloKills)

    medals = Medals(js['quickPlayStats']['awards'].get('medalsBronze'),
                      js['quickPlayStats']['awards'].get('medalsSilver'),
                      js['quickPlayStats']['awards'].get('medalsGold'))
    
    for hero in hero_list:
        print(hero)
        if hero in js['quickPlayStats']['careerStats']:
            elims = 0
            damageDone = 0
            deaths = 0
            soloKills = 0
            
            if js['quickPlayStats']['careerStats'][hero]['combat'] != None:
                
                if 'eliminations' in js['quickPlayStats']['careerStats'][hero]['combat']:
                    elims = js['quickPlayStats']['careerStats'][hero]['combat']['eliminations']

                if 'damageDone' in js['quickPlayStats']['careerStats'][hero]['combat']:
                    damageDone = js['quickPlayStats']['careerStats'][hero]['combat']['damageDone']

                if 'deaths' in js['quickPlayStats']['careerStats'][hero]['combat']:
                    deaths = js['quickPlayStats']['careerStats'][hero]['combat']['deaths']

                if 'soloKills' in js['quickPlayStats']['careerStats'][hero]['combat']:
                    soloKills = js['quickPlayStats']['careerStats'][hero]['combat']['soloKills']
                
            heroes.update({hero: Stats(elims,damageDone,deaths,soloKills)})
        else:
            heroes.update({hero: Stats(0,0,0,0)})
    
    return Player(js['name'], js['level'],js['rating'],js['prestige'], js['quickPlayStats']['games']['won'], qps, medals, heroes)

def df_object(p):
    item = [p.name,p.level,p.rating,p.prestige,p.games_won,p.qps.elims,p.qps.dmg_done,
            p.qps.deaths,p.qps.solo_kills,p.medals.bronze,p.medals.silver,p.medals.gold]
    
    for hero in hero_list:
        item.extend([p.hero[hero].elims,p.hero[hero].dmg_done,p.hero[hero].deaths,p.hero[hero].solo_kills])
        
    return item

In [5]:
usernames = pd.read_csv('../assets/data/usernames_scraped_fixed.csv')
usernames.head()
len(usernames['users'])

1676

In [13]:
##dataframe setup
columns = ['username','level','rating','prestige','games_won','qps_elims','qps_dmg_done',
           'qps_deaths','qps_solo_kills','medals_bronze','medals_silver','medals_gold']

for hero in hero_list:
    hero_data = [f'{hero}_elims',f'{hero}_dmg_done',f'{hero}_deaths',f'{hero}_solo_kills']
    columns.extend(hero_data)

data = pd.DataFrame(columns=columns)

amount = 0
for user in usernames['users'].values:
    url = f"https://ow-api.com/v1/stats/pc/us/{user}/complete"
    print(url)
    response = requests.get(url)
    j = json.loads(response.text)
    u = create_player(j)
    data.loc[len(data), :] = df_object(u)
    amount += 1
    
    percent = np.round((amount/len(usernames['users'])),decimals=2)
    clear_output()
    progress = widgets.IntProgress(
        value=amount,
        min=0,
        max=len(usernames['users'].values),
        step=1,
        description=f'{percent}%',
        bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
        orientation='horizontal'
    )
    display(progress)

IntProgress(value=1676, bar_style='info', description='1.0%', max=1676)

In [14]:
data.head()
data.tail()

df = pd.read_csv('user_info.csv')
print(df.shape)
df = df.append(data)
df.shape, data.shape

data.to_csv('user_info.csv',index=False)

(8, 136)


In [ ]:
# def s(username):
#     global search
#     search = username
    
# interactive(s, username='')

In [ ]:
# usernames = pd.read_csv('usernames_scraped_fixed.csv')
# usernames.head()

# df = pd.read_csv('usernames_scraped.csv')

In [ ]:
# username_scraped = []

# def str2bool(v):
#     return v.lower() in ("True", "true")

# for name in df['users']:
#     driver.get(f"https://playoverwatch.com/en-us/search?q={name}")
#     time.sleep(2)
#     page_source = driver.page_source
    
#     soup = BeautifulSoup(page_source)
#     players = soup.find_all('a', class_="player-badge")
    
#     for element in players:
#         locked = str2bool(element.find("div", {"data-visibility-private": True})['data-visibility-private'])
#         if(locked == False):
#             username_scraped.append(element.find(class_='player-badge-name').text.replace('#', '-'))
            
# print(len(username_scraped))

In [ ]:
# print(len(username_scraped))

# df1 = pd.read_csv('usernames_scraped_fixed.csv')

# df2 = pd.DataFrame(username_scraped,columns=['users'])

# df1 = df1.append(df2)

# df1.to_csv('usernames_scraped_fixed.csv',index=False)

# df1.shape

In [ ]:
# usernames['users'].values

In [ ]:
# def on_change(b):
#     global player
#     player = name=dropbox.value
#     print('player')

# dropbox = widgets.Select(
#     options=usernames['users'].values,
#     value=usernames['users'].values[0],
#     description='User:',
#     disabled=False
# )
# dropbox.observe(on_change, names='value')
        
# display(dropbox)

In [ ]:
# player

In [ ]:
# soup = BeautifulSoup(page_source)

# players = soup.find_all('a', class_="player-badge")
         
# def f(name):
#     return name

# def on_button_clicked(b):
#         global player
#         player = name=b.description

# displays = []
# for element in players:
#     locked = str2bool(element.find("div", {"data-visibility-private": True})['data-visibility-private'])
#     if(locked == True):
#         tooltip = 'Sorry, player has their profile set to private...'
#         icon = 'lock'
#     else:
#         tooltip = "Click to view this player"
#         icon = 'unlock'
#     button = widgets.Button(
#         description=element.find(class_='player-badge-name').text.capitalize().replace('#', '-'),
#         disabled=locked,
#         button_style='', # 'success', 'info', 'warning', 'danger' or ''
#         icon=icon,
#         tooltip=tooltip
#     )
#     out = widgets.Output()
        
#     button.on_click(on_button_clicked)
#     display(button,out)

In [ ]:
# url = f"https://ow-api.com/v1/stats/pc/us/{player}/complete"
# print(url)
# response = requests.get(url)

# print(response)

In [ ]:
# j = json.loads(response.text)
# if(j['private'] == True):
#     print("Sorry can't load this profile. it's private")
# else:
#     print(j['name'])